In [30]:
MODEL = '1'
BENCHMARKS_DIR = './train_sets/full_valid/' + MODEL
BENCHMARK_NAME = 'bass_pb40'
from transformers import CanineForSequenceClassification, CanineTokenizer, CanineConfig, Trainer, TrainingArguments, EarlyStoppingCallback
import torch
import pandas as pd
import os
from torch.utils.data import Dataset
from sklearn.metrics import roc_auc_score, average_precision_score, precision_recall_curve, roc_curve, confusion_matrix
import numpy as np
from scipy.special import softmax
import wandb
from imblearn.over_sampling import RandomOverSampler

## Model config

In [31]:
HIDDEN_SIZE = 128
NUM_HIDDEN_LAYERS = 1
NUM_ATTENTION_HEADS = 2
INTERMEDIATE_SIZE = 256
HIDDEN_DROPOUT_PROB = 0.5
ATTENTION_PROBS_DROPOUT_PROB = 0.5


model_config = CanineConfig(
    num_labels=2,
    hidden_size=HIDDEN_SIZE,
    num_hidden_layers=NUM_HIDDEN_LAYERS,
    num_attention_heads=NUM_ATTENTION_HEADS,
    intermediate_size=INTERMEDIATE_SIZE,
    hidden_dropout_prob=HIDDEN_DROPOUT_PROB,
    attention_probs_dropout_prob=ATTENTION_PROBS_DROPOUT_PROB,
    max_position_embeddings=42,
)

model = CanineForSequenceClassification(model_config)
model = model.to('cuda')

## Trainer config

In [32]:
NUM_TRAIN_EPOCHS = 20
LEARNING_RATE = 1e-4
PER_DEVICE_TRAIN_BATCH_SIZE = 32
PER_DEVICE_EVAL_BATCH_SIZE = 32
EVALUATION_STRATEGY = 'steps'
SAVE_STRATEGY = 'steps'
LOGGING_STEPS = 0.01
EVAL_STEPS = 0.04
SAVE_TOTAL_LIMIT = 6
SAVE_STEPS = 0.04

training_args = TrainingArguments(
    output_dir='./results',
    learning_rate=LEARNING_RATE,
    num_train_epochs=NUM_TRAIN_EPOCHS,
    per_device_train_batch_size=PER_DEVICE_TRAIN_BATCH_SIZE,
    per_device_eval_batch_size=PER_DEVICE_EVAL_BATCH_SIZE,
    evaluation_strategy=EVALUATION_STRATEGY,
    save_strategy=SAVE_STRATEGY,
    logging_steps=LOGGING_STEPS,
    eval_steps=EVAL_STEPS,
    save_steps=SAVE_STEPS,
    save_total_limit=SAVE_TOTAL_LIMIT,
    load_best_model_at_end=True,
    metric_for_best_model='eval_loss',
    greater_is_better=False,
    
)

In [33]:
tokenizer = CanineTokenizer(model_max_length=42)

train_set_file_path = os.path.join(BENCHMARKS_DIR, '%s.train.csv' % BENCHMARK_NAME)
valid_set_file_path = os.path.join(BENCHMARKS_DIR, '%s.valid.csv' % BENCHMARK_NAME)
test_set_file_path = os.path.join(BENCHMARKS_DIR, '%s.test.csv' % BENCHMARK_NAME)
train_set = pd.read_csv(train_set_file_path).dropna().drop_duplicates()
valid_set = pd.read_csv(valid_set_file_path).dropna().drop_duplicates()
test_set = pd.read_csv(test_set_file_path).dropna().drop_duplicates()

#ros = RandomOverSampler(sampling_strategy=0.05, random_state=0)
#train_encodings_resampled, train_labels_resampled = ros.fit_resample(train_set['seq'].values.reshape(-1, 1), train_set['label'])
#train_encodings_resampled_list = [item[0] for item in train_encodings_resampled.tolist()]


class AmDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx])
                for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)




encoded = tokenizer(train_encodings_resampled_list)

print('train set size: %d...' % len(train_set))
#train_dataset = AmDataset(encoded, train_labels_resampled.tolist())
train_dataset = AmDataset(tokenizer(train_set['seq'].tolist()), train_set['label'])
print('valid set size: %d...' % len(valid_set))
eval_dataset = AmDataset(tokenizer(valid_set['seq'].tolist()), valid_set['label'])
print('test set size: %d...' % len(test_set))
test_dataset = AmDataset(tokenizer(test_set['seq'].tolist()), test_set['label'])


train set size: 365179...
valid set size: 73047...
test set size: 73047...


In [34]:
def compute_metrics(eval_pred):
    """Compute AUROC, average precision, and recall at specified FPRs."""
    predictions, labels = eval_pred
    probabilities = softmax(predictions, axis=-1)  # Apply softmax to each row (sample)
    probabilities = probabilities[:, 1]  # Probability of positive class
    fpr, tpr, thresholds = roc_curve(labels, probabilities, pos_label=1)
  
    auroc = roc_auc_score(labels, probabilities)
    avg_prec = average_precision_score(labels, probabilities)

    # Precision and recall
    precision, recall, thresholds = precision_recall_curve(labels, probabilities)


    # Compute recall (TPR) at specified FPRs
    fprs = np.array([1e-1, 1e-2, 1e-3, 1e-4, 1e-5])
    recall_at_fprs = []
    for desired_fpr in fprs:
        # Find the TPR for this FPR
        # We're looking for the first threshold that gives a FPR <= our desired FPR
        try:
            threshold_index = next(i for i, current_fpr in enumerate(fpr) if current_fpr >= desired_fpr)
            recall_at_fprs.append(tpr[threshold_index])
        except StopIteration:
            recall_at_fprs.append(0.0)

    return {
        'auroc': auroc,
        'avg_prec': avg_prec,
        'recall_at_fprs': recall_at_fprs
    }

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=5)],
)

wandb.init(
    # set the wandb project where this run will be logged
    project="AmCANINE",
    
    # track hyperparameters and run metadata
    config={
    "learning_rate": LEARNING_RATE,
    "epochs": NUM_TRAIN_EPOCHS,
    "hidden_size": HIDDEN_SIZE,
    "num_hidden_layers": NUM_HIDDEN_LAYERS,
    "num_attention_heads": NUM_ATTENTION_HEADS,
    "intermediate_size": INTERMEDIATE_SIZE,
    }
)

trainer.train()
trainer.save_model('./results/best')

train/epoch,▁
train/global_step,▁
train/learning_rate,▁
train/loss,▁
train/epoch,0.2
train/global_step,2283
train/learning_rate,0.0001
train/loss,0.0262


c:\Users\krzys\anaconda3\envs\hf\lib\site-packages\transformers\optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(






















































































































































































































































































































































































c:\Users\krzys\anaconda3\envs\hf\lib\site-packages\wandb\sdk\wandb_run.py:2082: UserWarning: Run (4fpvx6sw) is finished. The call to `_console_raw_callback` will be ignored. Please make sure that you are using an active run.
  lambda data: self._console_raw_callback("stderr", data),
                      

{'loss': 0.034, 'learning_rate': 9.949964949176306e-05, 'epoch': 0.1}


































































































































































































































































































































































                                                       

  2%|▏         | 4565/228240 [16:17<2:01:40, 30.64it/s]



{'loss': 0.0167, 'learning_rate': 9.899929898352612e-05, 'epoch': 0.2}


...

































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































c:\Users

{'eval_loss': 0.011609325185418129, 'eval_auroc': 0.9781499458933864, 'eval_avg_prec': 0.5450644985393175, 'eval_recall_at_fprs': [0.9371428571428572, 0.7485714285714286, 0.4742857142857143, 0.37714285714285717, 0.18857142857142858], 'eval_runtime': 34.7239, 'eval_samples_per_second': 2103.653, 'eval_steps_per_second': 65.747, 'epoch': 4.0}












c:\Users\krzys\anaconda3\envs\hf\lib\site-packages\wandb\sdk\wandb_run.py:2082: UserWarning: Run (4fpvx6sw) is finished. The call to `_console_raw_callback` will be ignored. Please make sure that you are using an active run.
  lambda data: self._console_raw_callback("stderr", data),
                                                       

  2%|▏         | 4565/228240 [45:39<2:01:40, 30.64it/s] 




{'loss': 0.0093, 'learning_rate': 7.998597967052226e-05, 'epoch': 4.0}


KeyboardInterrupt: 

In [ ]:
# Load test set
test_set = pd.read_csv(test_set_file_path).dropna().drop_duplicates()
test_dataset = AmDataset(tokenizer(test_set['seq'].tolist()), test_set['label'])


# Evaluate on test set
#eval_results = trainer.evaluate(test_dataset)
#print(eval_results)

# confusion matrix
predictions, label_ids, metrics = trainer.predict(test_dataset)
probabilities = softmax(predictions, axis=-1)  # Apply softmax to each row (sample)
probabilities = probabilities[:, 1]  # Probability of positive class
threshold = 0.5
predictions = [1 if prob >= threshold else 0 for prob in probabilities]
cm = confusion_matrix(label_ids, predictions)
print(cm)

100%|██████████| 2283/2283 [00:29<00:00, 77.98it/s]


Computing metrics...
Best threshold according to Youden's Index: 0.0005499062826856971
[[72781    91]
 [   35   140]]


In [ ]:
model = CanineForSequenceClassification.from_pretrained('./results/canine_128_hs')

# print sum of model parameters
print('Number of model parameters: {:,}'.format(sum(p.numel() for p in model.parameters())))

# create confusion matrix
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)
predictions, label_ids, metrics = trainer.predict(test_dataset)
probabilities = softmax(predictions, axis=-1)  # Apply softmax to each row (sample)
probabilities = probabilities[:, 1]  # Probability of positive class
threshold = 0.5
predictions = [1 if prob >= threshold else 0 for prob in probabilities]
cm = confusion_matrix(label_ids, predictions)
print(cm)


Number of model parameters: 4,808,194


100%|██████████| 2283/2283 [00:30<00:00, 74.07it/s]

Computing metrics...
Best threshold according to Youden's Index: 0.0011606363113969564
[[72588   284]
 [   14   161]]
